## Two main types of Movie Recommendation Systems
1. Conten Based Recommendation System
2. Collaborative Filtering Recommendation System

## Workflow for this project
We are going to build a content-based movie recommendation system. 

The user enters his/ her favorite movie, then we'll recommendation others that he/ she might like.

Steps:
1. Data Collection
2. Data preprocessing
3. Feature extraction
4. User input
5. Cosine Similarity Algorithm
6. Result: List of movies

In [1]:
### importing the dependencies
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Data Collection and Pre-Processing

In [5]:
# loading the data from the csv file to a Pandas dataframe
movies_data = pd.read_csv('movies_details.csv')

In [6]:
# printing the first 5 rows of the dataframe
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [7]:
# show all column names
pd.set_option('display.max_columns', None)

In [8]:
movies_data[:1]

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron


In [9]:
# number of rows and columns in the dataframe
movies_data.shape

(4803, 24)

In [10]:
# selecting the relevant features for recommendation; aka feature selection
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']
selected_features

['genres', 'keywords', 'tagline', 'cast', 'director']

In [11]:
# replacing the missing values (aka null values) with null string
for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna('')

In [12]:
# combining all the 5 selected features
combined_features = movies_data['genres'] + ' ' + movies_data['keywords'] + ' ' + movies_data['tagline'] + ' ' + movies_data['cast'] + ' ' + movies_data['director']
combined_features

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object

In [13]:
# converting the text data to feature vectors

vectorizer = TfidfVectorizer()

In [14]:
feature_vectors = vectorizer.fit_transform(combined_features)

In [15]:
print(feature_vectors)

  (0, 2432)	0.17272411194153
  (0, 7755)	0.1128035714854756
  (0, 13024)	0.1942362060108871
  (0, 10229)	0.16058685400095302
  (0, 8756)	0.22709015857011816
  (0, 14608)	0.15150672398763912
  (0, 16668)	0.19843263965100372
  (0, 14064)	0.20596090415084142
  (0, 13319)	0.2177470539412484
  (0, 17290)	0.20197912553916567
  (0, 17007)	0.23643326319898797
  (0, 13349)	0.15021264094167086
  (0, 11503)	0.27211310056983656
  (0, 11192)	0.09049319826481456
  (0, 16998)	0.1282126322850579
  (0, 15261)	0.07095833561276566
  (0, 4945)	0.24025852494110758
  (0, 14271)	0.21392179219912877
  (0, 3225)	0.24960162956997736
  (0, 16587)	0.12549432354918996
  (0, 14378)	0.33962752210959823
  (0, 5836)	0.1646750903586285
  (0, 3065)	0.22208377802661425
  (0, 3678)	0.21392179219912877
  (0, 5437)	0.1036413987316636
  :	:
  (4801, 17266)	0.2886098184932947
  (4801, 4835)	0.24713765026963996
  (4801, 403)	0.17727585190343226
  (4801, 6935)	0.2886098184932947
  (4801, 11663)	0.21557500762727902
  (4801, 1672

## Cosine Similarity

In [16]:
# getting the similarity scores using cosine similarity
similarity = cosine_similarity(feature_vectors)

In [17]:
print(similarity)

[[1.         0.07219487 0.037733   ... 0.         0.         0.        ]
 [0.07219487 1.         0.03281499 ... 0.03575545 0.         0.        ]
 [0.037733   0.03281499 1.         ... 0.         0.05389661 0.        ]
 ...
 [0.         0.03575545 0.         ... 1.         0.         0.02651502]
 [0.         0.         0.05389661 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.02651502 0.         1.        ]]


In [18]:
similarity.shape

(4803, 4803)

## User input

In [19]:
# getting the movie name from the user
movie_name = input(' Enter your favorite movie name: ')

 Enter your favorite movie name: Tangled


## Output

In [20]:
# creating a list with all the movie names given in the dataset

list_of_all_titles = movies_data['title'].tolist()
list_of_all_titles[:3]

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre']

In [21]:
# finding the closest match for the movie name given by the user
close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
close_match

['Tangled', 'Triangle', 'Tango']

In [22]:
closest_match = close_match[0]
closest_match

'Tangled'

In [23]:
# find the index of the movie with title
index_of_the_movie = movies_data[movies_data['title'] == closest_match]['index'].values[0]
index_of_the_movie

6

In [24]:
# getting a list of similar movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))
similarity_score[:3]

[(0, 0.008237143013608844),
 (1, 0.014378790821108607),
 (2, 0.008427651382813465)]

In [25]:
len(similarity_score)

4803

In [26]:
# sorting the movies based on their simiilarity scores
sorted_similar_movies = sorted(similarity_score, key = lambda x : x[1], reverse = True)
sorted_similar_movies[:10]

[(6, 1.0000000000000002),
 (143, 0.19554086418770586),
 (81, 0.1944444529198599),
 (899, 0.17786202190372122),
 (1589, 0.15945340060382776),
 (578, 0.1529133936188724),
 (1804, 0.1432874748914728),
 (1404, 0.13630027751102275),
 (1695, 0.12920895168237598),
 (80, 0.12698468968156224)]

In [27]:
# print the name of similar movies based on the index
print('Movies suggested for you: \n')

i = 1

for movie in sorted_similar_movies:
    movie_index = movie[0]
    movie_title = movies_data[movies_data.index == movie_index]['title'].values[0]
    if i < 11: # print the top 10 movies
        print(i, ',', movie_title)
        i += 1

Movies suggested for you: 

1 , Tangled
2 , Pan
3 , Maleficent
4 , Shrek
5 , Racing Stripes
6 , Alvin and the Chipmunks: The Squeakquel
7 , Snow White: A Tale of Terror
8 , Street Fighter
9 , Aladdin
10 , Snow White and the Huntsman


## Sum up: Movie Recommendataion System

In [28]:
# combine everything in one cell
movie_name = input(' Enter your favorite movie name: ')

list_of_all_titles = movies_data['title'].tolist()

close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
closest_match = close_match[0]

index_of_the_movie = movies_data[movies_data['title'] == closest_match]['index'].values[0]
similarity_score = list(enumerate(similarity[index_of_the_movie]))
sorted_similar_movies = sorted(similarity_score, key = lambda x : x[1], reverse = True)

print('Movies suggested for you: \n')

i = 1

for movie in sorted_similar_movies:
    movie_index = movie[0]
    movie_title = movies_data[movies_data.index == movie_index]['title'].values[0]
    if i < 11: # print the top 10 movies
        print(i, ',', movie_title)
        i += 1

 Enter your favorite movie name: iron man
Movies suggested for you: 

1 , Iron Man
2 , Iron Man 2
3 , Iron Man 3
4 , Avengers: Age of Ultron
5 , The Avengers
6 , Captain America: Civil War
7 , Captain America: The Winter Soldier
8 , Ant-Man
9 , X-Men
10 , Made
